This didn't work for our project as each data response was too long.

In [47]:
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
import torch

In [48]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
src_model_name='Helsinki-NLP/opus-mt-en-fr'
src_tokenizer=MarianTokenizer.from_pretrained(src_model_name)
src_model=MarianMTModel.from_pretrained(src_model_name).to(device)

tgt_model_name='Helsinki-NLP/opus-mt-fr-en'
tgt_tokenizer=MarianTokenizer.from_pretrained(tgt_model_name)
tgt_model=MarianMTModel.from_pretrained(tgt_model_name).to(device)

In [55]:
def back_translate(text):
    inputs=src_tokenizer(text,return_tensors='pt',padding=True,truncation=True)
    translated_tokens=src_model.generate(**inputs)
    translated_text=src_tokenizer.decode(translated_tokens[0],skip_special_tokens=True)

    inputs=tgt_tokenizer(translated_text,return_tensors='pt',padding=True,truncation=True)
    back_translated_tokens=tgt_model.generate(**inputs)
    back_translated_text=tgt_tokenizer.decode(back_translated_tokens[0],skip_special_tokens=True)

    return back_translated_text

In [ ]:
trial='I am feeling not so good today. I live in Austin, I donot do much'
back_trial=back_translate(trial)
print('Original:',trial)
print('Augmented:',back_trial)

In [ ]:
def back_translate_batch(texts, src_tokenizer, src_model, tgt_tokenizer, tgt_model):
    # Translate from English to French
    inputs = src_tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = src_model.generate(**inputs)
    translated_texts = [src_tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]

    # Translate back from French to English
    inputs = tgt_tokenizer(translated_texts, return_tensors="pt", padding=True, truncation=True)
    back_translated_tokens = tgt_model.generate(**inputs)
    back_translated_texts = [tgt_tokenizer.decode(t, skip_special_tokens=True) for t in back_translated_tokens]

    return back_translated_texts

# Example usage for batch
texts=['I am feeling good today','The world is cruel']

augmented_texts = back_translate_batch(texts, src_tokenizer, src_model, tgt_tokenizer, tgt_model)
for original, augmented in zip(texts, augmented_texts):
    print(f"Original: {original}")
    print(f"Augmented: {augmented}")

In [63]:
aug_dataset=pd.read_csv(r'D:\Ematrix\DAIC\Preprocessed\aug_dataset.csv')
back_data=aug_dataset.loc[aug_dataset['PHQ8_Binary']==1]
texts=back_data['response'].to_list()

In [ ]:
augmented_texts = back_translate_batch(texts[1], src_tokenizer, src_model, tgt_tokenizer, tgt_model)
for original, augmented in zip(texts, augmented_texts):
    print(f"Original: {original}")
    print(f"Augmented: {augmented}")